# IDs & locations for 1 Mio wikipedia articles 
Source: Ben Dodson
- https://bendodson.com/weblog/2014/08/19/retiring-tubeupdates-and-wikilocation/
- https://bendodson.s3.amazonaws.com/wikilocation-dumps/index.html
- http://files.bendodson.com.s3.amazonaws.com/wikilocation-dumps/en.sql.gz

### Download the en.sql.gz file and convert to geopandas gdf

In [ ]:
# pip install pydeck pandas geopandas

In [22]:
import pandas as pd
import re
import csv
from io import StringIO
import pydeck as pdk

data = []

# Regular expression to match the lines with INSERT INTO statements for live_en table
insert_into_regex = re.compile(r"INSERT INTO `live_en` VALUES (.*?);")

# Read the file (assuming it's named 'en.sql')
with open('en.sql', 'r') as file:
    for line in file:
        match = insert_into_regex.search(line)
        if match:
            # Extract the values substring
            values_str = match.group(1)
            
            # Remove the leading and trailing parentheses for the entire INSERT INTO block
            values_str = values_str.strip("()")
            
            # Split the block into individual row strings
            row_strings = values_str.split("),(")
            
            # Process each row string
            for row_str in row_strings:
                # We need to handle the possibility of commas within the title field
                # Use csv.reader to parse the fields, using a StringIO object to treat the string as a file
                csv_reader = csv.reader(StringIO(row_str), delimiter=',', quotechar="'", doublequote=True, escapechar='\\')
                row_data = next(csv_reader)
                # Append the row data to our list
                data.append(row_data)

# Create a DataFrame with the extracted data
columns = ['id', 'lat', 'lng', 'type', 'title']
df = pd.DataFrame(data, columns=columns)

# Convert numeric columns to appropriate data types
df['id'] = df['id'].astype(int)
df['lat'] = df['lat'].astype(float)
df['lng'] = df['lng'].astype(float)

def clean_title(s, substring=", "):
    if s.startswith(substring):
        return s[len(substring):]
    else:
        return s

df["title"] = df["title"].apply(clean_title)

# if you'd like to add urls 
# df["url"] = df["id"].apply(lambda x: f"https://en.wikipedia.org/?curid={x}")

df

# df.to_parquet("wikipedia_locations_en_2014.parquet")

,id,lat,lng,type,title
0,39090650,31.7716,35.2030,landmark,"Billy Rose Art Garden, [[Israel Museum]] 194009"
1,39090650,31.7697,35.2142,landmark,Presidential Mansion 464725
2,39090650,31.7697,35.2142,landmark,Presidential Mansion 464728
3,39090650,32.0474,34.7660,landmark,"Abu Nabut Park, Derech Ben-Zvi 51 464729"
4,39090650,32.0474,34.7660,landmark,"Abu Nabut Park, Derech Ben-Zvi 51 464736"
...,...,...,...,...,...
1038005,32659976,55.6581,37.5811,,Zyuzino District
1038006,21823153,53.6667,21.3000,city,Zyzdrojowa Wola
1038007,21823153,53.6667,21.3000,city,Zyzdrojowa Wola
1038008,21823155,53.6667,21.2833,city,Zyzdrojowy Piecek


In [23]:
import geopandas as gpd 

gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_xy(df['lng'], df['lat']), crs="EPSG:4326")
del gdf["lat"]
del gdf["lng"]

gdf

# gdf.to_parquet("wikipedia_locations_en_2014_geo.parquet")

,id,type,title,geometry
0,39090650,landmark,"Billy Rose Art Garden, [[Israel Museum]] 194009",POINT (35.20300 31.77160)
1,39090650,landmark,Presidential Mansion 464725,POINT (35.21420 31.76970)
2,39090650,landmark,Presidential Mansion 464728,POINT (35.21420 31.76970)
3,39090650,landmark,"Abu Nabut Park, Derech Ben-Zvi 51 464729",POINT (34.76600 32.04740)
4,39090650,landmark,"Abu Nabut Park, Derech Ben-Zvi 51 464736",POINT (34.76600 32.04740)
...,...,...,...,...
1038005,32659976,,Zyuzino District,POINT (37.58110 55.65810)
1038006,21823153,city,Zyzdrojowa Wola,POINT (21.30000 53.66670)
1038007,21823153,city,Zyzdrojowa Wola,POINT (21.30000 53.66670)
1038008,21823155,city,Zyzdrojowy Piecek,POINT (21.28330 53.66670)


# Map Genration

In [ ]:
color_lookup = pdk.data_utils.assign_random_colors(df['type'])
df['color'] = df.apply(lambda row: color_lookup.get(row['type']), axis=1)

### Limit to N points

In [ ]:
MAX_POINTS = 100000

reduced_master_gdf = df.iloc[:MAX_POINTS]

view = pdk.data_utils.compute_view(reduced_master_gdf[["lng", "lat"]])

all_points = pdk.Layer(
    'ScatterplotLayer',     
    reduced_master_gdf,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    get_radius=3000,         
    opacity=0.9,
    get_fill_color="color",#[0,0,139],
    pickable=True
)

r = pdk.Deck(
    layers=[all_points],
    map_style="light"
)

r.to_html(f"pydeck_wikipedia_{MAX_POINTS}.html", notebook_display=True)

### All points (better don't display in Jupyter)

In [ ]:
view = pdk.data_utils.compute_view(df[["lng", "lat"]])

all_points = pdk.Layer(
    'ScatterplotLayer',     
    df,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    get_radius=1000,         
    opacity=0.9,
    get_fill_color="color",#[0,0,139],
    pickable=True
)

r = pdk.Deck(
    layers=[all_points],
    map_style="light"
)

r.to_html(f"pydeck_wikipedia.html", notebook_display=False)

In [ ]:
view = pdk.data_utils.compute_view(df[["lng", "lat"]])

all_points = pdk.Layer(
    'ScatterplotLayer',     
    df,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    get_radius=50,         
    opacity=0.9,
    get_fill_color="color",#[0,0,139],
    pickable=True
)

r = pdk.Deck(
    layers=[all_points],
    map_style="light"
)

r.to_html(f"pydeck_wikipedia_small_points.html", notebook_display=False)